In [7]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *
from packages.user_preference_prompt_by_persona.gernerate_preference import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast

import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown

In [8]:
def create_payload(name: str, email: str, password: str, displayName: str, user_attraction_tag_score: dict) -> dict:
    
    payload = {
        'username': name,
        'email': email,
        'password': password,
        'googleId': '',
        'displayName': displayName,
        "attractionTagScore": {
            "attractionTagFields": user_attraction_tag_score.copy()
        },
    }

    return payload.copy()

In [9]:
def getScorefromGeminiAPI(user_preference_prompt: list[dict]) -> dict:
    """
    Retrieves tag scores for a user based on their responses during the sign-up process using the Gemini API.

    Args:
        user_preference_prompt : A list of the user's responses to each question

    Returns:
        Dictionary representing scores for all tags.
    """

    # configure the Gemini API model
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model = genai.GenerativeModel('gemini-1.5-flash-latest')
    # model = genai.GenerativeModel('gemini-1.5-pro-latest')
    
    # construct the base prompt
    text_prompt = (
        "Based on the user's answers, generate a JSON string containing scores (0-1) for the following attributes (nothing else, no other sentences):"
        "\nExample JSON: "
        '{"Tourism":0,"Adventure":0,"Meditation":0,"Art":0,"Cultural":0,"Landscape":0,"Nature":0,"Historical":0,'
        '"Cityscape":0,"Beach":0,"Mountain":0,"Architecture":0,"Temple":0,"WalkingStreet":0,"Market":0,'
        '"Village":0,"NationalPark":0,"Diving":0,"Snuggle":0,"Waterfall":0,"Island":0,"Shopping":0,'
        '"Camping":0,"Fog":0,"Cycling":0,"Monument":0,"Zoo":0,"Waterpark":0,"Hiking":0,"Museum":0,'
        '"Riverside":0,"NightLife":0,"Family":0,"Kid":0,"Landmark":0,"Forest":0}'
    )

    # Add each question and its corresponding score to the prompt
    for item in user_preference_prompt:
        question = item["question"]
        score = item["score"]
        text_prompt += f"\nQuestion: {question}\nUser Answer: {score}/5"

    # 
    prompt = [text_prompt]

    # print token count for debugging
    print("total_tokens: ", model.count_tokens(prompt))
    
    # send the prompt to the Gemini API
    res_score_dict = {}
    try:
        response = model.generate_content(prompt)
        res_start_Idx = response.text.find('{')
        res_end_Idx = response.text.find('}')
        res_score_dict =  json.loads(response.text[res_start_Idx:res_end_Idx+1])

    except Exception as e:
        print("failed to use gemini api")
    
    return res_score_dict

In [10]:
# example result of user tagScore based on their responses on preference test
# user_preference_prompt = create_user_persona_art()


# user_attraction_tag_score = getScorefromGeminiAPI(user_preference_prompt=user_preference_prompt.copy())
# user_attraction_tag_score

In [11]:
# send POST API to create users
'''
20 user of following persona have been created:
- user_persona_nature --> love_art_{nth}
- user_persona_temple_historical --> love_temple_historical_{nth}
- user_persona_adventure --> love_adventure_{nth}
- user_persona_art --> love_art_{nth}

'''

for Idx in range(0, 20):
    cur_user_preference_prompt = create_user_persona_art()
    cur_user_attraction_tag_score = getScorefromGeminiAPI(user_preference_prompt=cur_user_preference_prompt.copy())
    
    cur_name = f"love_art_{Idx}"
    cur_email = f"love_art_{Idx}@gmail.com"
    cur_password = "qwerty1234"
    cur_displayName = f"love_art_dis_{Idx}"
 
    cur_payload = create_payload(
        name = cur_name, 
        email = cur_email, 
        password = cur_password, 
        displayName = cur_displayName,
        user_attraction_tag_score = cur_user_attraction_tag_score
    )

    try:
        API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/user/create"
        headers = {
            "Authorization": f"{os.environ['AUTH_BRANCH_API_KEY']}",
            "Content-Type": "application/json"
        }

        post_request = requests.post(url=API_ENDPOINT, json=cur_payload, headers=headers)
        print(post_request.text)

    except Exception as e:
        print("POST api failed at --> ", cur_payload['name'])
        print(e)
    

total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_0","email":"love_art_0@gmail.com","password":"$2a$10$O6ed4u1UlR5oHEHRSVwGzu5.er4fJAM8DJmqoNOfzG7dzrIpC3PxG","displayName":"love_art_0_4478","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.2,"Adventure":0,"Meditation":0.2,"Art":1,"Cultural":0.2,"Landscape":0.2,"Nature":0.2,"Historical":0.2,"Cityscape":0.2,"Beach":0,"Mountain":0.2,"Architecture":1,"Temple":0.2,"WalkingStreet":0.2,"Market":0.2,"Village":0,"NationalPark":0.2,"Diving":0,"Snuggle":0.2,"Waterfall":0.2,"Island":0,"Shopping":0.2,"Camping":0,"Fog":0.2,"Cycling":0,"Monument":0.2,"Zoo":0,"Waterpark":0,"Hiking":0,"Museum":0.2,"Riverside":0.2,"NightLife":0.2,"Family":0,"Kid":0,"Landmark":0.2,"Forest":0.2}},"_id":"677e544b519209b0aaff4fd6","createdAt":"2025-01-08T10:32:43.846Z","__v":0},"userRating":{"userId":"677e544b

In [12]:
'''
total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_0","email":"love_art_0@gmail.com","password":"$2a$10$P6YaRuY/q4TDHPA1Ziv7b.jZhfXQMBmHywIyjaRaBW/XZa3J9d3Q6","displayName":"love_art_0_3865","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.4,"Adventure":0.2,"Meditation":0,"Art":1,"Cultural":0.6,"Landscape":0.4,"Nature":0.4,"Historical":0.6,"Cityscape":0,"Beach":0.2,"Mountain":0.4,"Architecture":1,"Temple":0.6,"WalkingStreet":0,"Market":0,"Village":0.4,"NationalPark":0.4,"Diving":0.2,"Snuggle":0,"Waterfall":0.4,"Island":0.2,"Shopping":0,"Camping":0.2,"Fog":0,"Cycling":0.2,"Monument":0.6,"Zoo":0.4,"Waterpark":0.4,"Hiking":0.2,"Museum":0.6,"Riverside":0.4,"NightLife":0,"Family":0.4,"Kid":0.4,"Landmark":0.6,"Forest":0.4}},"_id":"677da874c82bd4de43ed6e96","createdAt":"2025-01-07T22:19:32.201Z","__v":0},"userRating":{"userId":"677da874c82bd4de43ed6e96","rating":[],"_id":"677da874c82bd4de43ed6e98","__v":0}}
total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_1","email":"love_art_1@gmail.com","password":"$2a$10$0spLd7chS8sTHXhG1vc5MO4Hz/asPOAOFysZP53x7YEVvl4W7ohu2","displayName":"love_art_1_7209","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.4,"Adventure":0.6,"Meditation":0.2,"Art":1,"Cultural":0,"Landscape":0.4,"Nature":0.2,"Historical":0,"Cityscape":0.4,"Beach":0.4,"Mountain":0.2,"Architecture":1,"Temple":0,"WalkingStreet":0.4,"Market":0.4,"Village":0.2,"NationalPark":0.2,"Diving":0.6,"Snuggle":0.2,"Waterfall":0.2,"Island":0.4,"Shopping":0.4,"Camping":0.6,"Fog":0.2,"Cycling":0.6,"Monument":0,"Zoo":0,"Waterpark":0,"Hiking":0.6,"Museum":0,"Riverside":0.2,"NightLife":0.4,"Family":0,"Kid":0,"Landmark":0,"Forest":0.2}},"_id":"677da877c82bd4de43ed6e9b","createdAt":"2025-01-07T22:19:35.823Z","__v":0},"userRating":{"userId":"677da877c82bd4de43ed6e9b","rating":[],"_id":"677da877c82bd4de43ed6e9d","__v":0}}
total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_2","email":"love_art_2@gmail.com","password":"$2a$10$2248/3wZqIilfrahaYKc9O6cbuo6Am1WqdJIdu1L7imaZxl.B.E4C","displayName":"love_art_2_1316","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.2,"Adventure":0,"Meditation":0.6,"Art":1,"Cultural":0,"Landscape":0.2,"Nature":0.2,"Historical":0,"Cityscape":0.2,"Beach":0,"Mountain":0.2,"Architecture":1,"Temple":0,"WalkingStreet":0.2,"Market":0.2,"Village":0,"NationalPark":0.2,"Diving":0,"Snuggle":0.6,"Waterfall":0.2,"Island":0,"Shopping":0.2,"Camping":0,"Fog":0.6,"Cycling":0,"Monument":0,"Zoo":0,"Waterpark":0,"Hiking":0,"Museum":0,"Riverside":0.2,"NightLife":0.2,"Family":0,"Kid":0,"Landmark":0,"Forest":0.2}},"_id":"677da87ac82bd4de43ed6ea0","createdAt":"2025-01-07T22:19:38.691Z","__v":0},"userRating":{"userId":"677da87ac82bd4de43ed6ea0","rating":[],"_id":"677da87ac82bd4de43ed6ea2","__v":0}}
total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_3","email":"love_art_3@gmail.com","password":"$2a$10$1.mzAsnKZpSuaN945RVgDeSPP2d5Ov9lUaB/3QrFZALZKh9VUzwAK","displayName":"love_art_3_4564","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.6,"Adventure":0,"Meditation":0,"Art":1,"Cultural":0.4,"Landscape":0.6,"Nature":0.6,"Historical":0.4,"Cityscape":0.6,"Beach":0,"Mountain":0.6,"Architecture":1,"Temple":0.4,"WalkingStreet":0.6,"Market":0.6,"Village":0.6,"NationalPark":0.6,"Diving":0,"Snuggle":0,"Waterfall":0.6,"Island":0,"Shopping":0.6,"Camping":0,"Fog":0,"Cycling":0,"Monument":0.4,"Zoo":0.2,"Waterpark":0.2,"Hiking":0,"Museum":0.4,"Riverside":0.6,"NightLife":0.6,"Family":0.2,"Kid":0.2,"Landmark":0.4,"Forest":0.6}},"_id":"677da87dc82bd4de43ed6ea5","createdAt":"2025-01-07T22:19:41.753Z","__v":0},"userRating":{"userId":"677da87dc82bd4de43ed6ea5","rating":[],"_id":"677da87dc82bd4de43ed6ea7","__v":0}}
total_tokens:  total_tokens: 532

{"message":"User and UserRating created successfully","user":{"username":"love_art_4","email":"love_art_4@gmail.com","password":"$2a$10$qnrXM8xZ78E78mLF06z/YOJd06lbmAJILpFOzODsl0Shni83uEtuy","displayName":"love_art_4_7421","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.6,"Adventure":0.4,"Meditation":0,"Art":0.8,"Cultural":0.6,"Landscape":0.6,"Nature":0.6,"Historical":0.6,"Cityscape":0.6,"Beach":0.2,"Mountain":0.6,"Architecture":0.8,"Temple":0.6,"WalkingStreet":0.6,"Market":0.6,"Village":0.6,"NationalPark":0.6,"Diving":0.4,"Snuggle":0,"Waterfall":0.6,"Island":0.2,"Shopping":0.6,"Camping":0.4,"Fog":0,"Cycling":0.4,"Monument":0.6,"Zoo":0.6,"Waterpark":0.6,"Hiking":0.4,"Museum":0.6,"Riverside":0.6,"NightLife":0.6,"Family":0.6,"Kid":0.6,"Landmark":0.6,"Forest":0.6}},"_id":"677da880c82bd4de43ed6eaa","createdAt":"2025-01-07T22:19:44.827Z","__v":0},"userRating":{"userId":"677da880c82bd4de43ed6eaa","rating":[],"_id":"677da880c82bd4de43ed6eac","__v":0}}
'''

'\ntotal_tokens:  total_tokens: 532\n\n{"message":"User and UserRating created successfully","user":{"username":"love_art_0","email":"love_art_0@gmail.com","password":"$2a$10$P6YaRuY/q4TDHPA1Ziv7b.jZhfXQMBmHywIyjaRaBW/XZa3J9d3Q6","displayName":"love_art_0_3865","imgPath":null,"firstName":null,"lastName":null,"points":0,"role":"user","favoritePlaces":[],"attractionTagScore":{"attractionTagFields":{"Tourism":0.4,"Adventure":0.2,"Meditation":0,"Art":1,"Cultural":0.6,"Landscape":0.4,"Nature":0.4,"Historical":0.6,"Cityscape":0,"Beach":0.2,"Mountain":0.4,"Architecture":1,"Temple":0.6,"WalkingStreet":0,"Market":0,"Village":0.4,"NationalPark":0.4,"Diving":0.2,"Snuggle":0,"Waterfall":0.4,"Island":0.2,"Shopping":0,"Camping":0.2,"Fog":0,"Cycling":0.2,"Monument":0.6,"Zoo":0.4,"Waterpark":0.4,"Hiking":0.2,"Museum":0.6,"Riverside":0.4,"NightLife":0,"Family":0.4,"Kid":0.4,"Landmark":0.6,"Forest":0.4}},"_id":"677da874c82bd4de43ed6e96","createdAt":"2025-01-07T22:19:32.201Z","__v":0},"userRating":{"user